Строим запрос к нашему сервису

In [21]:
import requests

import urllib.request
import json 

In [22]:
# формируем запрос
def send_json(x):
    title, text, subject = x
    body = {
        'title': title, 
        'text': text,
        'subject': subject
        }
    myurl = 'http://127.0.0.1:5000' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [23]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Сделаем N запросов и оценим время

In [24]:
N = 50

In [25]:
%%time
predictions = X_test[
                     ['title', 'text', 'subject']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: user 95.4 ms, sys: 14.7 ms, total: 110 ms
Wall time: 630 ms


In [26]:
predictions.values[:5]

array([0.99997858, 0.99997858, 0.99997858, 0.99997858, 0.99997858])

Посчитаем метрику

In [27]:
y_test.head()

,label
0,1
1,1
2,1
3,0
4,1


In [28]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.9999785781494986, F-Score=0.718, Precision=0.560, Recall=1.000


In [29]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.5

Анализ : В целом модель выполняем свои функции